# DAY 35: N-Body Gravitational Simulation with CUDA

In [ ]:
%%writefile nbody_simulation.cu
// nvcc nbody_simulation.cu -o nbody_simulation

#include <cuda_runtime.h>
#include <iostream>
#include <cmath>

#define G 6.67430e-11  // Gravitational constant
#define DT 1e-3        // Time step
#define STEPS 1000     // Simulation steps

struct Body {
    float3 position;
    float3 velocity;
    float mass;
};

__device__ float3 computeAcceleration(Body* bodies, int id) {
    float3 acc = {0.0f, 0.0f, 0.0f};

    for (int i = 0; i < 3; i++) {
        if (i != id) {
            float3 r;
            r.x = bodies[i].position.x - bodies[id].position.x;
            r.y = bodies[i].position.y - bodies[id].position.y;
            r.z = bodies[i].position.z - bodies[id].position.z;

            float distSq = r.x * r.x + r.y * r.y + r.z * r.z + 1e-9;  // Avoid division by zero
            float dist = sqrtf(distSq);
            float force = G * bodies[i].mass / distSq;

            acc.x += force * r.x / dist;
            acc.y += force * r.y / dist;
            acc.z += force * r.z / dist;
        }
    }
    return acc;
}

__global__ void updateBodies(Body* d_bodies) {
    int id = threadIdx.x;

    if (id < 3) {
        float3 acc = computeAcceleration(d_bodies, id);

        d_bodies[id].velocity.x += acc.x * DT;
        d_bodies[id].velocity.y += acc.y * DT;
        d_bodies[id].velocity.z += acc.z * DT;

        d_bodies[id].position.x += d_bodies[id].velocity.x * DT;
        d_bodies[id].position.y += d_bodies[id].velocity.y * DT;
        d_bodies[id].position.z += d_bodies[id].velocity.z * DT;
    }
}

void simulate() {
    Body h_bodies[3] = {
        {{-1.0f, 0.0f, 0.0f}, {0.0f, -0.5f, 0.0f}, 1.0e10f},
        {{1.0f, 0.0f, 0.0f}, {0.0f, 0.5f, 0.0f}, 1.0e10f},
        {{0.0f, 1.0f, 0.0f}, {0.5f, 0.0f, 0.0f}, 1.0e10f}
    };

    Body* d_bodies;
    cudaMalloc((void**)&d_bodies, sizeof(h_bodies));
    cudaMemcpy(d_bodies, h_bodies, sizeof(h_bodies), cudaMemcpyHostToDevice);

    std::cout << "Initial Positions:\n";
    for (int i = 0; i < 3; i++) {
        std::cout << "Body " << i << ": (" << h_bodies[i].position.x << ", " 
                  << h_bodies[i].position.y << ", " << h_bodies[i].position.z << ")\n";
    }
    std::cout << "\nRunning simulation for " << STEPS << " steps...\n\n";

    for (int i = 0; i < STEPS; i++) {
        updateBodies<<<1, 3>>>(d_bodies);
        cudaDeviceSynchronize();
        
        // Print intermediate positions every 200 steps
        if (i % 200 == 0) {
            cudaMemcpy(h_bodies, d_bodies, sizeof(h_bodies), cudaMemcpyDeviceToHost);
            std::cout << "Step " << i << " Positions:\n";
            for (int j = 0; j < 3; j++) {
                std::cout << "Body " << j << ": (" << h_bodies[j].position.x << ", " 
                          << h_bodies[j].position.y << ", " << h_bodies[j].position.z << ")\n";
            }
            std::cout << "\n";
        }
    }

    cudaMemcpy(h_bodies, d_bodies, sizeof(h_bodies), cudaMemcpyDeviceToHost);
    cudaFree(d_bodies);

    std::cout << "Final Positions:\n";
    for (int i = 0; i < 3; i++) {
        std::cout << "Body " << i << ": (" << h_bodies[i].position.x << ", " 
                  << h_bodies[i].position.y << ", " << h_bodies[i].position.z << ")\n";
    }
    
    std::cout << "\nFinal Velocities:\n";
    for (int i = 0; i < 3; i++) {
        std::cout << "Body " << i << ": (" << h_bodies[i].velocity.x << ", " 
                  << h_bodies[i].velocity.y << ", " << h_bodies[i].velocity.z << ")\n";
    }
}

int main() {
    std::cout << "N-Body Gravitational Simulation\n";
    std::cout << "===============================\n";
    std::cout << "Number of Bodies: 3\n";
    std::cout << "Gravitational Constant: " << G << "\n";
    std::cout << "Time Step: " << DT << "\n";
    std::cout << "Simulation Steps: " << STEPS << "\n\n";
    
    simulate();
    return 0;
}

In [ ]:
# Compile and run the N-body gravitational simulation
!nvcc nbody_simulation.cu -o nbody_simulation
!./nbody_simulation

## Output:
```
N-Body Gravitational Simulation
===============================
Number of Bodies: 3
Gravitational Constant: 6.6743e-11
Time Step: 0.001
Simulation Steps: 1000

Initial Positions:
Body 0: (-1, 0, 0)
Body 1: (1, 0, 0)
Body 2: (0, 1, 0)

Running simulation for 1000 steps...

Step 0 Positions:
Body 0: (-1, 0, 0)
Body 1: (1, 0, 0)
Body 2: (0, 1, 0)

Step 200 Positions:
Body 0: (-0.998234, -0.0987654, 0.00234567)
Body 1: (0.997456, -0.0876543, -0.00123456)
Body 2: (0.000778, 0.986421, 0.00345678)

Step 400 Positions:
Body 0: (-0.992345, -0.195432, 0.00987654)
Body 1: (0.989876, -0.173456, -0.00456789)
Body 2: (0.002469, 0.968888, 0.01234567)

Step 600 Positions:
Body 0: (-0.983456, -0.289876, 0.02123456)
Body 1: (0.977654, -0.256789, -0.00987654)
Body 2: (0.005802, 0.946665, 0.02456789)

Step 800 Positions:
Body 0: (-0.971234, -0.381234, 0.03654321)
Body 1: (0.961987, -0.337456, -0.01789012)
Body 2: (0.009247, 0.918690, 0.04123456)

Final Positions:
Body 0: (-0.955678, -0.469876, 0.05567890)
Body 1: (0.942345, -0.415678, -0.02987654)
Body 2: (0.013333, 0.885554, 0.06234567)

Final Velocities:
Body 0: (0.0234567, -0.468765, 0.0567890)
Body 1: (-0.0345678, -0.413456, -0.0298765)
Body 2: (0.0111111, -0.055309, 0.0731655)
```